In [ ]:
# import externals libraries
import os
import shutil
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

# import the needed OGGM modules
from oggm import cfg, utils, workflow
from oggm.core import gis, climate, flowline, vascaling

In [2]:
# define RGI IDs
rgi_ids = ['RGI60-11.00897']
tstar = None

# compute RGI region and version from RGI IDs
# assuming all they are all the same
rgi_region = (rgi_ids[0].split('-')[-1]).split('.')[0]
rgi_version = (rgi_ids[0].split('-')[0])[-2:]

# load default parameter file
cfg.initialize()

2020-07-21 09:47:58: oggm.cfg: Using configuration file: /Users/oberrauch/oggm-fork/oggm/params.cfg


In [3]:
# create working directory
wdir = '/Users/oberrauch/work/master/working_directories/'
wdir += 'hef_constants_wdir'
if not os.path.exists(wdir):
    os.makedirs(wdir)
# shutil.rmtree(wdir)
# os.makedirs(wdir)
# set path to working directory
cfg.PATHS['working_dir'] = wdir
# set RGI verion and region
cfg.PARAMS['rgi_version'] = rgi_version
# define how many grid points to use around the glacier,
# if you expect the glacier to grow large use a larger border
cfg.PARAMS['border'] = 80
# we use HistAlp climate data
cfg.PARAMS['baseline_climate'] = 'HISTALP'
# set the mb hyper parameters accordingly
cfg.PARAMS['prcp_scaling_factor'] = 1.75
cfg.PARAMS['temp_melt'] = -1.75
# the bias is defined to be zero during the calibration process,
# which is why we don't use it here to reproduce the results
cfg.PARAMS['use_bias_for_run'] = False

# read RGI entry for the glaciers as DataFrame
# containing the outline area as shapefile
rgidf = utils.get_rgi_glacier_entities(rgi_ids)

# get and set path to intersect shapefile
intersects_db = utils.get_rgi_intersects_region_file(region=rgi_region)
cfg.set_intersects_db(intersects_db)

# initialize the GlacierDirectory
gdirs = workflow.init_glacier_regions(rgidf)

# run gis tasks
workflow.gis_prepro_tasks(gdirs)
# run climate tasks
workflow.execute_entity_task(climate.process_histalp_data, gdirs)
workflow.execute_entity_task(climate.local_t_star, gdirs,
                             tstar=tstar, bias=0)
workflow.execute_entity_task(climate.mu_star_calibration, gdirs)
# run inversion tasks
workflow.inversion_tasks(gdirs)
# finalize preprocessing
workflow.execute_entity_task(flowline.init_present_time_glacier, gdirs)

2020-07-21 09:48:06: oggm.workflow: Execute entity task define_glacier_region on 1 glaciers
2020-07-21 09:48:06: oggm.workflow: Multiprocessing: using all available processors (N=4)
2020-07-21 09:48:06: oggm.core.gis: (RGI60-11.00897) define_glacier_region
2020-07-21 09:48:07: oggm.workflow: Execute entity task glacier_masks on 1 glaciers
2020-07-21 09:48:07: oggm.core.gis: (RGI60-11.00897) glacier_masks
2020-07-21 09:48:08: oggm.workflow: Execute entity task compute_centerlines on 1 glaciers
2020-07-21 09:48:08: oggm.core.centerlines: (RGI60-11.00897) compute_centerlines
2020-07-21 09:48:08: oggm.workflow: Execute entity task initialize_flowlines on 1 glaciers
2020-07-21 09:48:08: oggm.core.centerlines: (RGI60-11.00897) initialize_flowlines
2020-07-21 09:48:08: oggm.workflow: Execute entity task compute_downstream_line on 1 glaciers
2020-07-21 09:48:08: oggm.core.centerlines: (RGI60-11.00897) compute_downstream_line
2020-07-21 09:48:09: oggm.workflow: Execute entity task compute_downs

[None]

In [5]:
vascaling.get_scaling_constants(gdirs[0])

2020-07-21 09:48:52: oggm.utils: (RGI60-11.00897) glacier_statistics


(1.5038582086557708, 0.24399290770672957)

In [ ]:
utils.compile_glacier_statistics(gdirs).keys()

In [ ]:
glacier_stats = utils.compile_glacier_statistics(gdirs)
glacier_stats

In [ ]:
hef_stats = glacier_stats.loc['RGI60-11.00897']
hef_stats

In [ ]:
length = glacier_stats.loc['RGI60-11.00897']['longuest_centerline_km']*1e3
area = glacier_stats.loc['RGI60-11.00897']['rgi_area_km2']*1e6
volume = glacier_stats.loc['RGI60-11.00897']['inv_volume_km3']*1e9

In [ ]:
def compute_scaling_constant(volume, area, length):
    """Compute and return volume/length and volume/area scaling constant from given geometry"""
    c_l = volume/length**cfg.PARAMS['vas_q_length']
    c_a = volume/area**cfg.PARAMS['vas_gamma_area']
    return c_l, c_a

In [ ]:
c_l, c_a = compute_scaling_constant(volume, area, length)
print(c_l, c_a)

In [ ]:
glacier_stats = utils.glacier_statistics(gdirs[0])
glacier_stats

In [ ]:
cfg.PARAMS['vas_c_length_m']

In [ ]:
cfg.PARAMS['vas_c_area_m2']

In [ ]:
import pandas as pd
import xarray as xr

In [ ]:
ds = xr.open_dataset('/Users/oberrauch/work/master/data/eq_runs/hef_constmb_newscalingconst.nc')
ds_old = xr.open_dataset('/Users/oberrauch/work/master/data/eq_runs/hef_eq.nc')

In [ ]:
ds_old.sel(temp_bias=0.5, normalized=True, mb_model='constant').volume.plot.line(x='time')
ds.sel(temp_bias=0.5, normalized=True).volume.plot()
plt.gca().set_xlim(0,3000)

In [ ]:

ds_old.sel(temp_bias=0.5, normalized=True, mb_model='constant').volume.plot()

In [ ]:
cfg.initialize()

In [ ]:
area = 6.78e6
area_km2 = area*1e-6
volume = 0.191 * area**1.375
volume_km3 = 0.034 * area_km2**1.375
print(volume, volume_km3*1e9)

In [ ]:
cfg.PARAMS['vas_gamma_area']

In [ ]:
cfg.PARAMS['vas_c_area_m2']